In [1]:
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [2]:
res_id = '0505'

import geopandas as gpd
from pathlib import Path

# read the bounding box of the study area
val_pts = gpd.read_file(Path('../data/validation-locations/subset-validation-reservoirs-grand-pts.geojson'))
val_polys = gpd.read_file(Path('../data/validation-locations/subset-validation-reservoirs-grand.geojson'))

dumboor_pt = val_pts.loc[val_pts['tmsos_id']==res_id]
dumboor_poly = val_polys.loc[val_polys['tmsos_id']==res_id]

dumboor_poly.hvplot(geo=True, tiles='OSM', alpha=0.5) * dumboor_pt.hvplot(geo=True, color='red', size=200, alpha=0.5)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]

In [3]:
sat_area_fp = Path(f'../data/area/s2/{res_id}.csv')
sat_area = pd.read_csv(sat_area_fp, dtype={'reservoir': str}, parse_dates=['time'])
sat_area.hvplot(
    x='time', y='s2 area [km2]'
)

:Curve   [time]   (s2 area [km2])

In [4]:
dumboor_pt.columns

Index(['GRAND_ID', 'RES_NAME', 'DAM_NAME', 'ALT_NAME', 'RIVER', 'ALT_RIVER',
       'MAIN_BASIN', 'SUB_BASIN', 'NEAR_CITY', 'ALT_CITY', 'ADMIN_UNIT',
       'SEC_ADMIN', 'COUNTRY', 'SEC_CNTRY', 'YEAR', 'ALT_YEAR', 'REM_YEAR',
       'DAM_HGT_M', 'ALT_HGT_M', 'DAM_LEN_M', 'ALT_LEN_M', 'AREA_SKM',
       'AREA_POLY', 'AREA_REP', 'AREA_MAX', 'AREA_MIN', 'CAP_MCM', 'CAP_MAX',
       'CAP_REP', 'CAP_MIN', 'DEPTH_M', 'DIS_AVG_LS', 'DOR_PC', 'ELEV_MASL',
       'CATCH_SKM', 'CATCH_REP', 'DATA_INFO', 'USE_IRRI', 'USE_ELEC',
       'USE_SUPP', 'USE_FCON', 'USE_RECR', 'USE_NAVI', 'USE_FISH', 'USE_PCON',
       'USE_LIVE', 'USE_OTHR', 'MAIN_USE', 'LAKE_CTRL', 'MULTI_DAMS',
       'TIMELINE', 'COMMENTS', 'URL', 'QUALITY', 'EDITOR', 'LONG_DD', 'LAT_DD',
       'POLY_SRC', 'index_right', 'deltares_id', 'deltares_filename', 'db',
       'name', 'rid_id', 'grand_id', 'rid_filepath', 'resops_id',
       'rid_filename', 'tmsos_id', 'distance', 'geometry'],
      dtype='object')

In [5]:
deltares_id = int(dumboor_pt['deltares_id'].values[0])
deltares_name = f'{deltares_id:07}.csv'
deltares_fp = Path(f'../data/insitu/deltares/') / deltares_name
print(deltares_fp)

insitu_df = pd.read_csv(deltares_fp, parse_dates=['time'])
insitu_df['insitu area [km2]'] = insitu_df['area'] / 1e6
insitu_df

../data/insitu/deltares/0087711.csv


,time,area,insitu area [km2]
0,1988-01-11 03:47:00,3.800564e+07,38.005638
1,1988-02-28 03:48:00,3.396474e+07,33.964735
2,1988-03-31 03:48:00,3.047994e+07,30.479940
3,1988-09-23 03:49:00,4.290060e+07,42.900601
4,1988-10-09 03:49:00,4.314776e+07,43.147758
...,...,...,...
909,2021-09-23 04:41:00,3.909731e+07,39.097311
910,2021-09-23 04:42:00,3.902300e+07,39.023004
911,2021-09-25 04:32:00,3.852712e+07,38.527118
912,2021-09-25 04:32:00,3.837526e+07,38.375262


In [6]:
insitu_clean_df = insitu_df.groupby([insitu_df['time'].dt.date])[['area', 'insitu area [km2]']].mean().reset_index()
insitu_clean_df['time'] = pd.to_datetime(insitu_clean_df['time'])
insitu_clean_df

,time,area,insitu area [km2]
0,1988-01-11,3.800564e+07,38.005638
1,1988-02-28,3.396474e+07,33.964735
2,1988-03-31,3.047994e+07,30.479940
3,1988-09-23,4.290060e+07,42.900601
4,1988-10-09,4.314776e+07,43.147758
...,...,...,...
709,2021-08-31,3.731312e+07,37.313122
710,2021-09-05,3.836447e+07,38.364467
711,2021-09-23,3.906016e+07,39.060158
712,2021-09-25,3.845119e+07,38.451190


In [8]:
sat_area.hvplot(
    kind='scatter', x='time', y='s2 area [km2]', label='Sentinel-2 area [km2] (uncorrected for clouds)'
) * insitu_clean_df.loc[
    (insitu_clean_df['time'] >= sat_area['time'].min())&(insitu_clean_df['time'] <= sat_area['time'].max())
].hvplot(
    kind='scatter', x='time', y='insitu area [km2]', label='insitu area [km2]'
)

:Overlay
   .Scatter.Sentinel_hyphen_minus_2_area_left_square_bracket_km2_right_square_bracket_left_parenthesis_uncorrected_for_clouds_right_parenthesis :Scatter   [time]   (s2 area [km2])
   .Scatter.Insitu_area_left_square_bracket_km2_right_square_bracket                                                                            :Scatter   [time]   (insitu area [km2])

## todo: error metrics